# Classifying Urban sounds using Deep Learning

## 1 Data Exploration and Visualisation

### UrbanSound dataset

For this project we will use a dataset called Urbansound8K. The dataset contains 8732 sound excerpts (<=4s) of urban sounds from 10 classes, which are:

- Air Conditioner
- Car Horn
- Children Playing
- Dog bark
- Drilling
- Engine Idling
- Gun Shot
- Jackhammer
- Siren
- Street Music

The accompanying metadata contains a unique ID for each sound excerpt along with it's given class name.

A sample of this dataset is included with the accompanying git repo and the full dataset can be downloaded from [here](https://urbansounddataset.weebly.com/urbansound8k.html).

### Audio sample file data overview

These sound excerpts are digital audio files in .wav format. 

Sound waves are digitised by sampling them at discrete intervals known as the sampling rate (typically 44.1kHz for CD quality audio meaning samples are taken 44,100 times per second). 

Each sample is the amplitude of the wave at a particular time interval, where the bit depth determines how detailed the sample will be also known as the dynamic range of the signal (typically 16bit which means a sample can range from 65,536 amplitude values). 

Therefore, the data we will be analysing for each sound excerpts is essentially a one dimensional array or vector of amplitude values. 

### Analysing audio data 

For audio analysis, we will be using the following libraries: 

#### 1. IPython.display.Audio 

This allows us to play audio directly in the Jupyter Notebook. 

#### 2. Librosa 

librosa is a Python package for music and audio processing by Brian McFee and will allow us to load audio in our notebook as a numpy array for analysis and manipulation. 

You may need to install librosa using pip as follows: 

`pip install librosa'     

### Auditory inspection 

We will use `IPython.display.Audio` to play the audio files so we can inspect aurally. 

In [ ]:
import IPython.display as ipd

ipd.Audio('../audio/1-977-A-39.wav')

### Visual inspection

We will load a sample from each class and visually inspect the data for any patterns. We will use librosa to load the audio file into an array then librosa.display and matplotlib to display the waveform. 

### Observations 


From a visual inspection we can see that it is tricky to visualise the difference between some of the classes. 

Particularly, the waveforms for reptitive sounds for air conditioner, drilling, engine idling and jackhammer are similar in shape.  

Likewise the peak in the dog barking sample is simmilar in shape to the gun shot sample (albeit the samples differ in that there are two peaks for two gunshots compared to the one peak for one dog bark). Also, the car horn is similar too. 

There are also similarities between the children playing and street music. 

The human ear can naturally detect the difference between the harmonics, it will be interesting to see how well a deep learning model will be able to extract the necessary features to distinguish between these classes. 


However, it is easy to differentiate from the waveform shape, the difference between certain classes such as dog barking and jackhammer. 

In [ ]:
# Class: Air Conditioner

filename = '../audio/1-977-A-39.wav'
plt.figure(figsize=(12,4))
data,sample_rate = librosa.load(filename)
_ = librosa.display.waveplot(data,sr=sample_rate)
ipd.Audio(filename)

In [34]:
# Load imports

import IPython.display as ipd
import librosa
import librosa.display
import matplotlib.pyplot as plt

### Dataset Metadata 

Here we will load the UrbanSound metadata .csv file into a Panda dataframe. 

In [37]:
import pandas as pd
metadata = pd.read_csv('/home/animesh/ESC-50-master/meta/esc50.csv')
metadata.head(10)
metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80 entries, 0 to 79
Data columns (total 7 columns):
filename    80 non-null object
fold        80 non-null int64
target      80 non-null int64
category    80 non-null object
esc10       80 non-null bool
src_file    80 non-null int64
take        80 non-null object
dtypes: bool(1), int64(3), object(3)
memory usage: 3.9+ KB


### Class distributions

In [ ]:
print(metadata.category.value_counts())

In [39]:
metadata.loc[metadata['target'] != 24,'target'] = 2
metadata.loc[metadata['target'] == 24,'target'] = 1


In [40]:
count=0
for index in metadata.target:
    if index == 1:
        count+=1
print(count)

40


### Observations 

Here we can see the Class labels are unbalanced. Although 7 out of the 10 classes all have exactly 1000 samples, and siren is not far off with 929, the remaining two (car_horn, gun_shot) have significantly less samples at 43% and 37% respectively. 

This will be a concern and something we may need to address later on. 

### Audio sample file properties

Next we will iterate through each of the audio sample files and extract, number of audio channels, sample rate and bit-depth. 

In [ ]:
# Load various imports 
import pandas as pd
import os
import librosa
import librosa.display

from helpers.wavfilehelper import WavFileHelper
wavfilehelper = WavFileHelper()

audiodata = []
for index, row in metadata.iterrows():
    
    #file_name = os.path.join(os.path.abspath('/home/animesh/ESC-50-master/audio/'),'fold'+str(row["fold"])+'/',str(row["file_name"]))
    file_name = os.path.join(os.path.abspath('/home/animesh/ESC-50-master/audio/'),str(row["filename"]))
    print(file_name)
    data = wavfilehelper.read_file_properties(file_name)
    audiodata.append(data)

# Convert into a Panda dataframe
audiodf = pd.DataFrame(audiodata, columns=['num_channels','sample_rate','bit_depth'])

### Audio channels 

Most of the samples have two audio channels (meaning stereo) with a few with just the one channel (mono). 

The easiest option here to make them uniform will be to merge the two channels in the stero samples into one by averaging the values of the two channels. 

In [42]:
# num of channels 

print(audiodf.num_channels.value_counts(normalize=True))

1    1.0
Name: num_channels, dtype: float64


### Sample rate 

There is a wide range of Sample rates that have been used across all the samples which is a concern (ranging from 96k to 8k).

This likley means that we will have to apply a sample-rate conversion technique (either up-conversion or down-conversion) so we can see an agnostic representation of their waveform which will allow us to do a fair comparison. 

In [43]:
# sample rates 

print(audiodf.sample_rate.value_counts(normalize=True))

44100    1.0
Name: sample_rate, dtype: float64


### Bit-depth 

There is also a wide range of bit-depths. It's likely that we may need to normalise them by taking the maximum and minimum amplitude values for a given bit-depth. 

In [44]:
# bit depth

print(audiodf.bit_depth.value_counts(normalize=True))

16    1.0
Name: bit_depth, dtype: float64


### Other audio properties to consider 

We may also need to consider normalising the volume levels (wave amplitude value) if this is seen to vary greatly, by either looking at the peak volume or the RMS volume. 

### *In the next notebook we will preprocess the data*

In [45]:
metadata.head()

,filename,fold,target,category,esc10,src_file,take
0,1-11687-A-47.wav,1,2,airplane,False,11687,A
1,1-12653-A-15.wav,1,2,water_drops,False,12653,A
2,1-12654-A-15.wav,1,2,water_drops,False,12654,A
3,1-12654-B-15.wav,1,2,water_drops,False,12654,B
4,1-13571-A-46.wav,1,2,church_bells,False,13571,A
